In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import wget
from pathlib import Path
from os import environ

In [10]:
root = Path(':'.join(environ.get("PYTHONPATH", ".").split(":")[:2]))
data = root / 'data' / 'MO'

In [12]:
df = pd.read_csv("../data/MO/MO MI images - imagesAndNames.csv")
df = df.dropna()
df = df[df['name'].str.split(' ').str.len() > 1]
df

,image,name,created,license,rightsHolder
0,https://mushroomobserver.org/images/640/2.jpg,Xylaria magnoliae,2004-07-17,http://creativecommons.org/licenses/by-sa/3.0/,Nathan Wilson
1,https://mushroomobserver.org/images/640/16.jpg,Volvopluteus gloiocephalus,2005-01-07,http://creativecommons.org/licenses/by-sa/3.0/,Nathan Wilson
2,https://mushroomobserver.org/images/640/26.jpg,Panellus stipticus,2004-11-26,http://creativecommons.org/licenses/by-sa/3.0/,Nathan Wilson
3,https://mushroomobserver.org/images/640/36.jpg,Sutorius eximius,2004-07-14,http://creativecommons.org/licenses/by-sa/3.0/,Nathan Wilson
4,https://mushroomobserver.org/images/640/37.jpg,Harrya chromapes,1994-09-15,http://creativecommons.org/licenses/by-sa/3.0/,Nathan Wilson
...,...,...,...,...,...
254915,https://mushroomobserver.org/images/640/131432...,Amanita velosa,2021-03-10,http://creativecommons.org/licenses/by-sa/3.0/,James Baker
254916,https://mushroomobserver.org/images/640/131432...,Amanita velosa,2021-03-10,http://creativecommons.org/licenses/by-sa/3.0/,James Baker
254917,https://mushroomobserver.org/images/640/131437...,Laricifomes officinalis,2021-03-10,http://creativecommons.org/licenses/by-sa/3.0/,-Josh
254918,https://mushroomobserver.org/images/640/131437...,Laricifomes officinalis,2021-03-10,http://creativecommons.org/licenses/by-sa/3.0/,-Josh


In [13]:
(data / 'images').mkdir(exist_ok=True)

In [18]:
df = df.replace(f'^(https?://)([^/]+)/images(/.+)', r'\1images.\2\3', regex=True)

In [19]:
!wget {df['image'].values[0]}

SYSTEM_WGETRC = c:/progra~1/wget/etc/wgetrc
syswgetrc = C:\Program Files (x86)\GNUWin32/etc/wgetrc
--2023-12-06 06:35:54--  https://images.mushroomobserver.org/640/2.jpg
Resolving images.mushroomobserver.org... 204.93.192.148
Connecting to images.mushroomobserver.org|204.93.192.148|:443... connected.
OpenSSL: error:1407742E:SSL routines:SSL23_GET_SERVER_HELLO:tlsv1 alert protocol version
Unable to establish SSL connection.


In [ ]:
df.to_csv(data / 'MO_images.csv', index=False)

In [14]:
wget.download(df['image'].values[0], out=data / 'images')

HTTPError: HTTP Error 403: Forbidden